# `innvestigate` demo

## setup

In [1]:
import os

import keras
import keras.backend
import keras.models

import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.tests.networks.base
import innvestigate.utils.visualizations as ivis

import numpy as np
import pandas as pd
import tensorflow as tf

import plotly.graph_objs as go
import plotly.offline

plotly.offline.init_notebook_mode(connected=True)

Using TensorFlow backend.


## can we do linear models?

check out the 15-or-so line model developed on [the `tensorflow` "getting started" page](https://www.tensorflow.org/tutorials/) -- this is a fully linear model on mnist (could be iris too)

and instead of the `mnist` dataset, let's look at the `boston_housing` dataset (there's [a whole separate tutorial page for that](https://www.tensorflow.org/tutorials/keras/basic_regression)

In [2]:
# mnist = tf.keras.datasets.mnist

# (x_train, y_train),(x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

housing = tf.keras.datasets.boston_housing
(train_data, train_labels), (test_data, test_labels) = housing.load_data()

# shuffle
order = np.argsort(np.random.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]

# normalize
train_mean = train_data.mean(axis=0)
train_std = train_data.std(axis=0)

train_data = (train_data - train_mean) / train_std
test_data = (test_data - train_mean) / train_std

In [3]:
train_data.shape

(404, 13)

In [4]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
                'TAX', 'PTRATIO', 'B', 'LSTAT']

df = pd.DataFrame(train_data, columns=column_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.397253,1.412057,-1.126646,-0.256833,-1.027385,0.726354,-1.000164,0.023834,-0.511142,-0.047533,-1.490674,0.415841,-0.836487
1,0.087846,-0.483615,1.028326,-0.256833,1.371293,-3.817250,0.676891,-1.049006,1.675886,1.565287,0.784476,-0.000885,-0.775760
2,-0.395379,1.201427,-0.690066,-0.256833,-0.942023,0.827918,-0.939245,0.259915,-0.626249,-0.914123,-0.398602,0.448077,-1.347144
3,-0.403759,3.097099,-1.022279,-0.256833,-1.095675,0.351129,-1.480347,2.364762,-0.626249,-0.330379,-0.262093,0.400521,-0.880652
4,-0.348692,-0.483615,-0.720935,-0.256833,-0.455458,3.467186,0.501302,-0.417158,-0.165822,-0.595170,-0.489608,0.289557,-1.119419


In [5]:
def build_model():
    model = keras.Sequential([
        keras.layers.Dense(64, activation=tf.nn.relu, 
                           input_shape=(train_data.shape[1],)),
        keras.layers.Dense(64, activation=tf.nn.relu),
        keras.layers.Dense(1)
    ])
  
    optimizer = tf.train.RMSPropOptimizer(0.001)
  
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae'])
    return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [6]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        if epoch % 100 == 0:
            print('')
        print('.', end='')

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
        
history = model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    validation_split=0.2,
    verbose=0,
    callbacks=[early_stop, PrintDot()]
)


....................................................................................................
.....................................................................

In [7]:
data = [
    go.Scatter(
        x=history.epoch,
        y=history.history['mean_absolute_error'],
        name='train loss'
    ),
    go.Scatter(
        x=history.epoch,
        y=history.history['val_mean_absolute_error'],
        name='val loss'
    )
]

plotly.offline.iplot(data)

## building the `innvestigate` analyzers

In [8]:
innvestigate.create_analyzer?

In [9]:
patternnet = innvestigate.analyzer.PatternNet(model)
patternnet.fit(train_data)

In [10]:
df_test = pd.DataFrame(test_data, columns=column_names)
df_test.loc[:, 'true_label'] = test_labels
df_test.loc[:, 'predicted_label'] = model.predict(test_data).flatten()
df_test = df_test.sort_values(by='predicted_label')
df_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,true_label,predicted_label
69,2.307977,-0.483615,1.028326,-0.256833,1.157888,-0.395089,1.110488,-1.061040,1.675886,1.565287,0.784476,0.448077,1.936245,5.6,8.460787
35,2.042626,-0.483615,1.028326,-0.256833,1.217641,-1.787370,0.734226,-1.095759,1.675886,1.565287,0.784476,0.448077,2.656686,7.4,9.377044
0,1.553694,-0.483615,1.028326,-0.256833,1.038381,0.235458,1.110488,-0.939769,1.675886,1.565287,0.784476,-3.484596,2.250921,7.2,9.759588
52,0.848688,-0.483615,1.028326,-0.256833,1.217641,-1.736587,1.002985,-0.971677,1.675886,1.565287,0.784476,0.448077,1.785808,9.7,11.806353
79,1.147281,-0.483615,1.028326,-0.256833,1.217641,-1.956644,1.110488,-1.060695,1.675886,1.565287,0.784476,0.192956,2.467605,10.2,11.883493


In [11]:
df_patternnet_test = pd.DataFrame(
    patternnet.analyze(df_test[column_names].values),
    columns=column_names
)
df_patternnet_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.494229,0.757712,-0.773128,0.364250,-0.663545,0.950013,-0.713535,0.633953,-0.656933,-0.717212,-0.797962,0.348614,-1.000000
1,-0.620794,0.845150,-0.997091,0.335678,-0.846692,0.853374,-0.896216,0.816736,-0.938127,-1.000000,-0.949180,0.455002,-0.996042
2,-0.581429,0.793902,-0.879050,0.410994,-0.774231,0.859073,-0.792342,0.726885,-0.881090,-0.929299,-0.911844,0.518682,-1.000000
3,-0.577043,0.837570,-0.931218,0.368029,-0.784374,0.886926,-0.848229,0.763503,-0.889633,-0.941791,-0.969495,0.387462,-1.000000
4,-0.589467,0.833927,-0.947409,0.352645,-0.804875,0.873625,-0.859513,0.777063,-0.909623,-0.960847,-0.955852,0.410733,-1.000000


all test records, sorted by *predicted* label, components then visualized as a heatmap

In [12]:
data = [
    go.Heatmap(
        x=column_names,
        z=df_patternnet_test.values,
        colorscale='RdBu'
    )
]

layout = go.Layout(
    height=1200
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure)

it might be as useful to look at the kde / histograms for each category:

In [13]:
fixedbins = {
    'start': -1,
    'end': 1,
    'size': 0.2
}
data = [
    go.Histogram(
        x=df_patternnet_test[col].values,
        xbins=fixedbins,
        opacity=0.6,
        name=col
    )
    for col in column_names
]

layout = go.Layout(barmode='overlay')
figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure)

from this it's pretty clear that `RM` is a strong predictor, `LSTAT` is a strong (anti-correlated) predictor, and `CHAS` is not strongly correlated

In [14]:
data = [go.Scatter(
    x=df_test.RM,
    y=df_test.predicted_label,
    mode='markers'
)]

layout = go.Layout(
    xaxis={'title': 'RM'},
    yaxis={'title': 'predicted_label'},
    title='correlation between RM and predicted label for DNN model'
)
figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure)

In [15]:
data = [go.Scatter(
    x=df_test.LSTAT,
    y=df_test.predicted_label,
    mode='markers'
)]

layout = go.Layout(
    xaxis={'title': 'LSTAT'},
    yaxis={'title': 'predicted_label'},
    title='correlation between LSTAT and predicted label for DNN model'
)
figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure)

In [16]:
keras.backend.clear_session()

In [17]:
tf.reset_default_graph()